# Problem 1

## Use Logistic Regression With Mean Squared Error Loss

In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
import math
import numpy as np

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def sigmoid(z):
    return 1 / (1 + math.exp(-z))


def softmax(z, z_list):
    z_vec = np.array(z_list)
    denom = np.sum(np.exp(z_vec))
    num = np.exp(z)
    return np.divide(num,denom)


def sgd_mse(a, y, x, w, b, lr):
    a_prime = 1 - a
    w_gradient = ((a - y) * a_prime) * x
    b_gradient = (a - y) * a_prime
    w_update = w - lr * w_gradient
    b_update = b - lr * b_gradient
    return w_update, b_update


def sgd_bce(a, y, x, w, b, lr):
    w_gradient = (a - y) * x
    b_gradient = a - y
    w_update = w - lr * w_gradient
    b_update = b - lr * b_gradient
    return w_update, b_update


def sgd_cce(a, y, x, w, b, lr, j):
    gradient = 0
    w_gradient = 0
    b_gradient = 0
    for k in range(10):
        if k == j:
            k_delta = 1
        else:
            k_delta = 0
        gradient = gradient + (y[k] * (a - k_delta))
    w_gradient = gradient * x
    b_gradient = gradient
    w_update = w - lr * w_gradient
    b_update = b - lr * b_gradient
    return w_update, b_update


def preprocess_image(image):
    whitespace_regions = np.zeros((30, 30))
    img_len = img_width = 28
    num_black_pixels = 0
    for i in range(img_len):
        for j in range(img_width):
            if image[i][j] > 50:
                image[i][j] = 255
            else:
                image[i][j] = 0
                num_black_pixels += 1
                whitespace_regions[i + 1][j + 1] = 1
    return image, whitespace_regions, num_black_pixels


def connected_components(image):
    image, whitespace_regions, num_black_pixels = preprocess_image(image)
    need_to_visit = whitespace_regions
    num_whitespace_regions = 1
    i = j = 1
    visit_stack = []
    while np.any(need_to_visit):
        #Initial case
        if need_to_visit[i][j]:
            need_to_visit[i][j] = 0
            visit_stack.append((i,j))
        #Down-Right
        elif need_to_visit[i+1][j+1]:
            i = i + 1
            j = j + 1
            need_to_visit[i][j] = 0
            visit_stack.append((i,j))
        #Right
        elif need_to_visit[i][j+1]:
            j = j + 1
            need_to_visit[i][j] = 0
            visit_stack.append((i,j))
        #Up-Right
        elif need_to_visit[i-1][j+1]:
            i = i - 1
            j = j + 1
            need_to_visit[i][j] = 0
            visit_stack.append((i, j))
        #Up
        elif need_to_visit[i-1][j]:
            i = i - 1
            need_to_visit[i][j] = 0
            visit_stack.append((i, j))
        #Up-Left
        elif need_to_visit[i-1][j-1]:
            i = i - 1
            j = j - 1
            need_to_visit[i][j] = 0
            visit_stack.append((i, j))
        #Left
        elif need_to_visit[i][j-1]:
            j = j - 1
            need_to_visit[i][j] = 0
            visit_stack.append((i, j))
        #Down-Left
        elif need_to_visit[i+1][j-1]:
            i = i + 1
            j = j - 1
            need_to_visit[i][j] = 0
            visit_stack.append((i, j))
        #Down
        elif need_to_visit[i+1][j]:
            i = i + 1
            need_to_visit[i][j] = 0
            visit_stack.append((i, j))
        else:
            if visit_stack:
                i,j = visit_stack.pop()
            else:
                num_whitespace_regions += 1
                for k in range(30):
                    for w in range(30):
                        if need_to_visit[k][w]:
                            i,j = k,w
    return num_whitespace_regions, num_black_pixels

In [10]:
def logistic_regression_sigmoid():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    y_train_mat = [[], [], [], [], [], [], [], [], [], []]

    for label in y_train:
        y_train_mat[label].append(1)
        for i in range(10):
            if i != label:
                y_train_mat[i].append(0)

    for i in range(10):
        y_train_mat[i] = np.array(y_train_mat[i])

    x_train = x_train.reshape((60000, 28 * 28))
    x_train = x_train.astype('float32') / 255

    #x_test = x_test.reshape((10000, 28 * 28))
    #x_test = x_test.astype('float32') / 255

    w = [0] * 784
    w = np.array(w)
    b = [0]
    b = np.array(b)

    lr = .001

    classifiers = [0] * 10
    
    y_hat = ground_truth = ''

    # Train k classifiers
    for k in range(10):
        # Train for 1 epoch
        for epoch in range(1):
            # Train with batch size of 1
            for i in range(y_train_mat[k].shape[0]):
                x = x_train[i]
                z = np.dot(x, w) + b
                a = sigmoid(z)

                y = y_train_mat[k][i]
                
                if y:
                    ground_truth = k
                else:
                    ground_truth = 'Not k'
                
                if a >= .5:
                    is_digit_prediction = True
                    y_hat = k
                else:
                    is_digit_prediction = False
                    y_hat = 'Not k'

                w, b = sgd_mse(a, y, x, w, b, lr)
                if i % 1000 == 0:
                    print("Epoch {0}: Predicted: {1}, Ground Truth: {2}".format(epoch, y_hat, ground_truth))

if __name__ == '__main__':
    logistic_regression_sigmoid()

Epoch 0: Predicted: 0, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: 0
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 0, Ground Truth: 0
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Gr

Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 3, Ground Truth: 3
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 3, Ground Truth: 3
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 3, Ground Truth: 3
Epoch 0: Predicted: Not k, Ground Tru

Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 6, Ground Truth: 6
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: 6
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 6, Ground Truth: 6
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground

Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Pred

# Problem 2

## Use Logistic Regression With Binary Cross Entropy Loss

In [3]:
def logistic_regression_sigmoid():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    y_train_mat = [[], [], [], [], [], [], [], [], [], []]

    for label in y_train:
        y_train_mat[label].append(1)
        for i in range(10):
            if i != label:
                y_train_mat[i].append(0)

    for i in range(10):
        y_train_mat[i] = np.array(y_train_mat[i])

    x_train = x_train.reshape((60000, 28 * 28))
    x_train = x_train.astype('float32') / 255

    #x_test = x_test.reshape((10000, 28 * 28))
    #x_test = x_test.astype('float32') / 255

    w = [0] * 784
    w = np.array(w)
    b = [0]
    b = np.array(b)

    lr = .001

    classifiers = [0] * 10
    
    y_hat = ground_truth = ''

    # Train k classifiers
    for k in range(10):
        # Train for 1 epoch
        for epoch in range(1):
            # Train with batch size of 1
            for i in range(y_train_mat[k].shape[0]):
                x = x_train[i]
                z = np.dot(x, w) + b
                a = sigmoid(z)

                y = y_train_mat[k][i]
                
                if y:
                    ground_truth = k
                else:
                    ground_truth = 'Not k'
                
                if a >= .5:
                    is_digit_prediction = True
                    y_hat = k
                else:
                    is_digit_prediction = False
                    y_hat = 'Not k'

                w, b = sgd_bce(a, y, x, w, b, lr)
                if i % 1000 == 0:
                    print("Epoch {0}: Predicted: {1}, Ground Truth: {2}".format(epoch, y_hat, ground_truth))

if __name__ == '__main__':
    logistic_regression_sigmoid()

Epoch 0: Predicted: 0, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: 0
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 0, Ground Truth: 0
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Gr

Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 3, Ground Truth: 3
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 3, Ground Truth: 3
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Gr

Epoch 0: Predicted: Not k, Ground Truth: 6
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 6, Ground Truth: 6
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k

Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 9, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: 9, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: 9
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: Not k
Epoch 0: Predicted: Not k, Ground Truth: 9
Epoch 0: Predicted: Not k, Gr

# Problem 3

- Load the training and test data using Keras, no validation set needed.
- Create network with an input layer consisting of 28 x 28 input neurons and an output layer consisting of 10 output neurons.
- Use softmax and categorical cross entropy loss.
- Implement mini-batch stochastic gradient descent using only numpy, that is, you are not allowed to use TensorFlow/Keras for SGD.

In [ ]:
def logistic_regression_softmax():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    y_train = to_categorical(y_train)

    x_train = x_train.reshape((60000, 28 * 28))
    x_train = x_train.astype('float32') / 255

    #x_test = x_test.reshape((10000, 28 * 28))
    #x_test = x_test.astype('float32') / 255

    w = [[0] * 784, [0] * 784,[0] * 784,[0] * 784,[0] * 784,
         [0] * 784, [0] * 784,[0] * 784,[0] * 784,[0] * 784]

    b = [0] * 10

    lr = .001

    for epoch in range(1):
        # Train with batch size of 1
        for i in range(y_train.shape[0]):
            z = []
            a = []
            x = x_train[i]
            for neuron in range(10):
                z.append(np.dot(x, w[neuron]) + b[neuron]) 
            for neuron in range(10):
                a.append(softmax(z[neuron], z))

            y_hat = np.argmax(a)
            y_arg_max = np.argmax(y_train[i])

            if y_hat == y_arg_max:
                is_digit_prediction = True
            else:
                is_digit_prediction = False

            for j in range(10):
                w[j], b[j] = sgd_cce(a[j], y_train[i], x, w[j], b[j], lr, j)
            
            if i % 1000 == 0:
                print("Epoch {0}: Predicted: {1}, Ground Truth: {2}".format(epoch, y_hat, y_arg_max))
if __name__ == '__main__':
    logistic_regression_softmax()

Epoch 0: Predicted: 0, Ground Truth: 5
Epoch 0: Predicted: 0, Ground Truth: 0
Epoch 0: Predicted: 6, Ground Truth: 5
Epoch 0: Predicted: 0, Ground Truth: 9
Epoch 0: Predicted: 7, Ground Truth: 7
Epoch 0: Predicted: 7, Ground Truth: 7
Epoch 0: Predicted: 6, Ground Truth: 6
Epoch 0: Predicted: 8, Ground Truth: 8
Epoch 0: Predicted: 0, Ground Truth: 0
Epoch 0: Predicted: 2, Ground Truth: 6
Epoch 0: Predicted: 3, Ground Truth: 3
Epoch 0: Predicted: 6, Ground Truth: 6
Epoch 0: Predicted: 7, Ground Truth: 7
Epoch 0: Predicted: 7, Ground Truth: 7
Epoch 0: Predicted: 1, Ground Truth: 1
Epoch 0: Predicted: 5, Ground Truth: 5
Epoch 0: Predicted: 8, Ground Truth: 8
Epoch 0: Predicted: 3, Ground Truth: 3
Epoch 0: Predicted: 4, Ground Truth: 4
Epoch 0: Predicted: 8, Ground Truth: 8
Epoch 0: Predicted: 5, Ground Truth: 5
Epoch 0: Predicted: 9, Ground Truth: 7
Epoch 0: Predicted: 3, Ground Truth: 3
Epoch 0: Predicted: 7, Ground Truth: 7
Epoch 0: Predicted: 8, Ground Truth: 8
Epoch 0: Predicted: 7, Gr

# Problem 4

## Reimplement the Network from Problem 3 Entirely in Keras.

In [5]:
def logistic_regression_keras():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    y_train = to_categorical(y_train)

    x_train = x_train.reshape((60000, 28 * 28))
    x_train = x_train.astype('float32') / 255

    model = Sequential()
    model.add(Dense(784, input_shape=(784,)))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='sgd', loss='mse', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=1, batch_size=1)
if __name__ == '__main__':
    logistic_regression_keras()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/1
60000/60000 [==============================] - 229s - loss: 0.0212 - acc: 0.8691   


# Problem 5

## Extend the Network from Problem 4 by Adding New Features.

In [6]:
def logistic_regression_keras_modified():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    white_space_regions = []
    num_black_pixels = []
    percent = 0
    for i in range(60000):
        white_space_region_value, black_pixels_count = connected_components(x_train[i])
        white_space_regions.append(white_space_region_value / 3.0)
        num_black_pixels.append(black_pixels_count / 784.0)
        if i % 1000 == 0:
            print("{0}% Connected Components Preprocessing Completed".format(percent))
            percent = i / 60000.0

    y_train = to_categorical(y_train)

    x_train = x_train.reshape((60000, 28 * 28))
    x_train = x_train.astype('float32') / 255
    
    x_train = np.column_stack((x_train, white_space_regions))
    x_train = np.column_stack((x_train, num_black_pixels))
    model = Sequential()
    model.add(Dense(786, input_shape=(786,)))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='sgd', loss='mse', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=1, batch_size=1)
if __name__ == '__main__':
    logistic_regression_keras_modified()

0% Connected Components Preprocessing Completed
0.0% Connected Components Preprocessing Completed
0.0166666666667% Connected Components Preprocessing Completed
0.0333333333333% Connected Components Preprocessing Completed
0.05% Connected Components Preprocessing Completed
0.0666666666667% Connected Components Preprocessing Completed
0.0833333333333% Connected Components Preprocessing Completed
0.1% Connected Components Preprocessing Completed
0.116666666667% Connected Components Preprocessing Completed
0.133333333333% Connected Components Preprocessing Completed
0.15% Connected Components Preprocessing Completed
0.166666666667% Connected Components Preprocessing Completed
0.183333333333% Connected Components Preprocessing Completed
0.2% Connected Components Preprocessing Completed
0.216666666667% Connected Components Preprocessing Completed
0.233333333333% Connected Components Preprocessing Completed
0.25% Connected Components Preprocessing Completed
0.266666666667% Connected Component